In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob
import os

from music21 import converter, instrument, note, chord, stream

from keras.layers import Input, Dense, Reshape, Dropout, Bidirectional, LSTM
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model, model_from_json
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model, model_from_json

Using TensorFlow backend.


In [2]:
base_path = "Project/"
training_dir = "data/Training/"
output_base_path = "output/"
base_model_path = "models/"
gan_model_result = "gan_model_result/"

os.makedirs(base_path + output_base_path, exist_ok=True)
os.makedirs(base_path + output_base_path + gan_model_result, exist_ok=True)

os.makedirs(base_path + base_model_path, exist_ok=True)
os.makedirs(base_path + base_model_path + gan_model_result, exist_ok=True)

In [3]:
def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in glob.glob(base_path + training_dir + "*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

In [4]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab)/2) / (float(n_vocab)/2)
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [5]:
def generate_notes(model, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)
    
    # Get pitch names and store in a dictionary
    pitchnames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = numpy.append(pattern,index)
        #pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [6]:
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

In [7]:
class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_discriminator(self):

        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)

        return Model(noise, seq)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load and convert the data
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Training the model
        for epoch in range(epochs):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)

            # Print the progress and save into loss lists
            if epoch % sample_interval == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
              self.disc_loss.append(d_loss[0])
              self.gen_loss.append(g_loss)
        
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        # Get pitch names and store in a dictionary
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        
        # Use random noise to generate sequences
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes = [int_to_note[int(x)] for x in pred_notes]
        
        create_midi(pred_notes, base_path + output_base_path + gan_model_result + 'gan_final')
        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

In [8]:
gan = GAN(rows=100)    
gan.train(epochs=5, batch_size=32, sample_interval=1)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1024)              4198400   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dens

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon BlackWhite - Ns Castle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon BlackWhite - Ns Farewell.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon BlackWhite - Nuvema Town.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon BlackWhite - Opelucid City White.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon BlackWhite - Pokemon League.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon BlackWhite - Relic Song.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon BlackWhite - Route 10.mid
Parsing C:/Users/chinm/Documents/Courses/Se

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon DiamondPearlPlatinum - Team Galactic Grunt Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon DiamondPearlPlatinum - Torn WorldDistortion World.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon DiamondPearlPlatinum - Twinleaf Town Day.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon DiamondPearlPlatinum - Veilstone City.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon DiamondPearlPlatinum - Villa  Twinleaf Music Box.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon DiamondPearlPlatinum - WiFi Lobby.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon HeartGoldSoulSilver - Pokemaniac Encounter.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon HeartGoldSoulSilver - Pokemon Center.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon HeartGoldSoulSilver - PokeWalker Synchronisation.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon HeartGoldSoulSilver - Route 4748.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon HeartGoldSoulSilver - Safari Zone Gate.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon HeartGoldSoulSilver - Team Rocket Hideout.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/T

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - Temporal Tower.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - The Gatekeepers.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - Time Gear Remix.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - Title Theme.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Mystery Dungeon Gates to Infinity - Desperation Light Arrangement.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Mystery Dungeon Gates to 

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RubySapphireEmerald - Dewford Town.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RubySapphireEmerald - Elite 4 Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RubySapphireEmerald - Ending Theme.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RubySapphireEmerald - Evergrande City.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RubySapphireEmerald - Fallarbor Town.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RubySapphireEmerald - Fortree City.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RubySapphi

C:\Users\chinm\Anaconda\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Instructions for updating:
Use tf.cast instead.
0 [D loss: 0.697486, acc.: 0.00%] [G loss: 0.690606]
1 [D loss: 0.684617, acc.: 57.81%] [G loss: 0.689037]
2 [D loss: 0.667867, acc.: 67.19%] [G loss: 0.691588]
3 [D loss: 0.638372, acc.: 68.75%] [G loss: 0.700332]
4 [D loss: 0.584130, acc.: 71.88%] [G loss: 0.725003]


In [9]:
# Save Model
model_json = gan.combined.to_json()

with open(base_path + base_model_path + gan_model_result + "gan_combined.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
gan.combined.save_weights(base_path + base_model_path + gan_model_result + "gan_combined.h5")
print("Saved model to disk")


Saved model to disk


In [10]:
# Load Model

# load json and create model
json_file = open(base_path + base_model_path + gan_model_result +'gan_combined.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights(base_path + base_model_path + gan_model_result + "gan_combined.h5")
print("Loaded model from disk")

Loaded model from disk


In [11]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
model_2 (Model)              (None, 100, 1)            1022820   
_________________________________________________________________
model_1 (Model)              (None, 1)                 5907457   
Total params: 6,930,277
Trainable params: 6,926,693
Non-trainable params: 3,584
_________________________________________________________________


In [12]:
# Save Model

model_json = gan.generator.to_json()
with open(base_path + base_model_path + gan_model_result + "gan_generator.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
gan.generator.save_weights(base_path + base_model_path + gan_model_result + "gan_generator.h5")
print("Saved model to disk")


Saved model to disk


In [13]:
# Load Model

# load json and create model
json_file = open(base_path + base_model_path + gan_model_result + 'gan_generator.json', 'r')
loaded_model_gen_json = json_file.read()
json_file.close()
loaded_model_gen = model_from_json(loaded_model_gen_json)
# load weights into new model
loaded_model_gen.load_weights(base_path + base_model_path + gan_model_result + "gan_generator.h5")
print("Loaded model from disk")

Loaded model from disk


In [14]:
loaded_model_gen.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 100, 1)            1022820   
Total params: 1,022,820
Trainable params: 1,019,236
Non-trainable params: 3,584
_________________________________________________________________


In [15]:
# Save Model

model_json = gan.discriminator.to_json()
with open(base_path + base_model_path + gan_model_result + "gan_discriminator.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
gan.discriminator.save_weights(base_path + base_model_path + gan_model_result + "gan_discriminator.h5")
print("Saved model to disk")


Saved model to disk


In [16]:
# Load Model

# load json and create model
json_file = open(base_path + base_model_path + gan_model_result + 'gan_discriminator.json', 'r')
loaded_model_disc_json = json_file.read()
json_file.close()
loaded_model_disc = model_from_json(loaded_model_disc_json)
# load weights into new model
loaded_model_disc.load_weights(base_path + base_model_path + gan_model_result + "gan_discriminator.h5")
print("Loaded model from disk")

Loaded model from disk


In [17]:
loaded_model_disc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 1)            0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 5907457   
Total params: 5,907,457
Trainable params: 5,907,457
Non-trainable params: 0
_________________________________________________________________


In [18]:
out = loaded_model_gen.predict(np.random.normal(0, 2, (32, 1000)))

In [19]:
notes = get_notes()

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon - Farewell, Pikachu!.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon - Lavender town.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon - Littleroot Town.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon - Lugias Song.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon - Oracion.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon - Pallet Town.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon - Pkmn Elite 4.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon - pokece

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Colosseum - Relic Forest.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Colosseum - Semifinal Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Colosseum - The Under.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon DiamondPearlPlatinum - Approaching Champion Cynthia.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon DiamondPearlPlatinum - Azelf Mesprit and Uxie Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon DiamondPearlPlatinum - Battle Champion.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Diamond

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon GoldSilverCrystal - Johto Gym Leader Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon GoldSilverCrystal - Johto Wild Pokemon Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon GoldSilverCrystal - Kanto Gym Leader Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon GoldSilverCrystal - Lavender Town.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon GoldSilverCrystal - Menu.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon GoldSilverCrystal - National Park.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon 

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Mystery Dungeon BlueRed Rescue Team - Title Screen.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - A Sinister Smell Team Skulls Theme.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - A Wish For Peace.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - Aegis Cave.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - Amp Plains.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon Mystery Dungeon Explorers

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RedBlueYellow - Elite Four.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RedBlueYellow - Ending Theme.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RedBlueYellow - Evolution.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RedBlueYellow - Game Corner Theme.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RedBlueYellow - Gym Leader Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RedBlueYellow - Gym.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RedBlueYellow - Hall of Fame.mid
Parsing C:/Users/chinm/D

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RubySapphireEmerald - Route 111 Desert.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RubySapphireEmerald - Route 120.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RubySapphireEmerald - Slateport City.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RubySapphireEmerald - Sootopolis City.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RubySapphireEmerald - Surfing.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RubySapphireEmerald - Trainers School.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training\Pokemon RubySapphireE

In [20]:
gan.generate(notes)